In [22]:

import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from enum import Enum, auto

from random import uniform, random
from math import cos, sin, pi, inf, sqrt
import os


# スクリプトのあるディレクトリパス（ここは適宜書き換えてください）.
this_dir = "C:\\Users\\na366\\Desktop\\Python\\SIR"
# 粒子半径.
radius = 0.075
# 時間刻み.
dt = 0.01
# 正方形 Box の一辺の長さの半分.
half = 5
# 速度の大きさ.
speed = 1
# 回復にかかる時間（回復率 γ の逆数）.
time_to_recover = 2
# バッファ.buff = 300
buff = 1

# 図.
fig = plt.figure(figsize=plt.figaspect(1/1))

# 図の背景色は黒.
fig.set_facecolor('black')


class State(Enum):
    Susceptible = auto()        # S 連番で、1、未感染
    Infected = auto()           # I 連番で、2、感染
    Recovered = auto()          # R 連番で、3、回復

class Person:

    def __init__( self, co, state, standstill ):
        
        self.co = co
        self.coords = np.array([self.co])    #  ([]) ⇒ [] 配列
        self.arg = uniform(-pi, pi)         # 乱数区間
        self.standstill = standstill         # 停止
        
        if self.standstill:           # True [1],[a]
            #                          停止させるのであれば初期速度は (0.0, 0.0).
            vel = np.array([0.0, 0.0])
        else:                         # Falus [0],[None]                                   # Falus　とは [0],[None]
            #                         そうでないのであれば偏角
            #                         乱数（self.arg）方向に大きさ speed で等速運動.
            vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])

        self.vel = vel
        self.state = state
        self.infection_start = inf        # 無限大(inf, -inf) = (+∞ , -∞)

    def move( self ):
        
        """
        もし self が停止しているのならば, 動かす. > line231_call
        """
        if self.standstill:           # True [1],[a]
            self.vel = np.array([speed * cos(self.arg), speed * sin(self.arg)])


class Population:
    
    def __init__( self, number, comeback, p=None ):
        
        self.number = number
        self.R0 = (radius * (sqrt(2) * speed) * time_to_recover * self.number)/ (half ** 2)
        # 
        print("R0: %f" % self.R0)
        self.persons = []
        self.p = p
        init_coords = []
        
        if self.p is not None:
            self.Re = (1.0 - self.p) * (1.0 + (sqrt(2) - 1) * self.p) * self.R0
            print("Re: %f" % self.Re)
            
        # 総人口数分の人間を生成.     number=500,[0,1,---,499],繰返,代入
        for i in range(self.number):
            if i == 0:
                # 最初の人間は感染者として初期条件を設定.
                # 初期座標は原点に設定.
                init_co = np.array([0.0, 0.0])
                
                # class Person 呼出　と　class State 呼出か？
                infected = Person(co=init_co,
                                  state=State.Infected,
                                  standstill=False)
                # class_insstance⇒infested
                infected.infection_start = 0
                self.persons.append(infected)
            else:
                # それ以外の人間は未感染者として初期条件を設定.
                while True:
                    
                    # 以前の人間の初期座標とは異なるまで while ループで座標をランダムに生成.
                    init_co = np.array([uniform(-half, half), uniform(-half, half)])
                    # 適切な
                    appropriate = True
                    for pre_init_co in init_coords:
                        #                          L2ノルム
                        dist = np.linalg.norm(init_co - pre_init_co, ord=2)
                        
                        # 以前の人間の初期座標との距離が 3 *（半径）の時は不適合とする.
                        # 最低 2 *（半径）で判定しないと, 人間を表す粒子が重なる.
                        # この判定距離を大きく取りすぎると, 人口数が大きいとき
                        # 新規座標がいつまで経っても不適合となりうることに注意.
                        if dist <= 3 * radius:
                            appropriate = False
                    if appropriate:
                        break
                # 行動制限の割合（p）を指定していない時は全ての人間を動かす.
                # そうでない場合は, 一様ランダムに 0.0 ~ 1.0 の数を生成し,
                # その値が行動制限の割合以下ならば停止させ, そうでない時は動かす.
                if self.p is None:
                    standstill = False        # 動かす
                elif random() < self.p:
                    standstill = True         # 停止
                else:
                    standstill = False        # 動かす
                    
                self.persons.append(Person(co=init_co,
                                           state=State.Susceptible,
                                           standstill=standstill))
            init_coords.append(init_co)
            
        self.times = np.array([0])
        self.zeros = np.array([0])
        self.I = np.array([1])
        self.S = np.array([self.number - 1])
        self.R = np.array([0])
        self.comeback = comeback
        
        print("self.persons=",self.persons,"\n")
        print("init_coords=",init_coords,"\n")

    def simulate_sir( self, threshold=2 ):
        # 時間の index.
        i = 0
        # 感染者数が閾値（threshold）を超えたか否か.
        exceeded_threshold = False
        # 行動制限を解除したか否か.
        comebacked = False
        # 感染が収束した時の時間のインデックス.
        subsided = None
        
        # 感染が収束してから、バッファの分だけ、時間のインデックスが進むまで、
        # SIR モデルのシミュレーションを行う.
        
        while True:
            # 各人間に対し, 感染が始まってから回復時間経った時には,
            # その人間の感染状態を免疫獲得状態に変更する.
            # そして各人間を移動させる.
            # 正方形 Box の壁に当たった場合には反射させる.
            
            for person in self.persons:
                if (i - person.infection_start) * dt > time_to_recover:
                    person.state = State.Recovered
                # 速度の時間刻み dt のスカラー倍分進む.
                person.co += dt * person.vel
                # x 軸方向での反射.
                if abs(person.co[0]) > half:
                    person.co[0] = np.sign(person.co[0]) * half
                    person.vel[0] *= -1
                # y 軸方向での反射.
                if abs(person.co[1]) > half:
                    person.co[1] = np.sign(person.co[1]) * half
                    person.vel[1] *= -1
                    
            # 索引,index
            indices = list(range(self.number))        # [0,1,,,,499]
            if i==0:
                print("182_i=",i,"indices=",indices," len=",len(indices),"\n")
                
            while len(indices) > 0:                # ()の,長さ,要素数[500]
                k = indices.pop(0)
                print("\n186_i=",i,"k=",k,"  indices=",indices,"len(indices)=",len(indices))
                person = self.persons[k]
                print("187_i=",i,"k=",k,",_self.persons[k] =",person,"\n")
                                
                for m in indices:                # indices= [0,1,,,,499]
                    
                    other = self.persons[m]
                    print("193_self.persons[",m,"]",other)
                    vec = person.co - other.co
                    print("195_person.co",person.co,"other.co=",other.co,"\n")
                    dist = np.linalg.norm(vec, ord=2)    # L2
                    
                    # person と other の距離が 2 *（半径）より小さいとき接触したと判定.
                    # 簡単のため 3 人以上の同時接触は考えない.
                    
                    if 0 < dist < 2 * radius:
                        normal = vec / dist
                        # person と other が重ならないようにするための補正（correction）.
                        correction = (radius - (dist / 2)) * normal
                        person.co += correction
                        other.co -= correction
                        # 物理的な完全弾性衝突の場合は -= np.dot(person.vel - other.vel, normal) * normal.
                        person.vel -= (2 * np.dot(person.vel, normal)) * normal
                        # 物理的な完全弾性衝突の場合は -= np.dot(other.vel - person.vel, normal) * normal.
                        other.vel -= (2 * np.dot(other.vel, normal)) * normal
                        indices.remove(l)
                        # 感染状態を必要であれば変化させる.
                        if person.state == State.Infected and other.state == State.Susceptible:
                            other.state = State.Infected
                            other.infection_start = i
                        elif person.state == State.Susceptible and other.state == State.Infected:
                            person.state = State.Infected
                            person.infection_start = i
                        break
                        
            # 時間の index を一つ進める.
            i += 1
            # 総感染者数.
            I = 0
            # 総未感染者数.
            S = 0
            # 総免疫獲得者数.
            R = 0
            
            for person in self.persons:
                # 各人間に対し, 現在の座標を追加記憶させる.
                person.coords = np.vstack([person.coords, person.co])
                
                # print("person.coords=",person.coords)
                
                if person.state == State.Infected:
                    I += 1
                elif person.state == State.Susceptible:
                    S += 1
                elif person.state == State.Recovered:
                    R += 1
                    
            # もし総感染者数 I が感染者数の閾値（threshold）を
            # 初めて超えた場合, exceeded_threshold を真 Ture とする.
            #                     超えた_閾値
            #     閾値   < I ×   not(Ture )
            if threshold < I and not exceeded_threshold:
                exceeded_threshold = True
                
            # 行動制限を解除する場合（self.comeback が真の場合）,
            #  条件１，２，３、ならば、４、へ
            # １、もしまだ、行動制限を解除しておらず（comebacked が偽）,
            # ２、総感染者数 I が感染者数の閾値（threshold）を既に超えており
            #                                  （exceeded_threshold が真）
            # ３、その後総感染者数 I が感染者数の閾値（threshold）を下回った場合,
            # ４、停止させている人間は全て動かす.

            if I <= threshold and self.comeback and not comebacked and exceeded_threshold:
                for person in self.persons:
                    person.move()
                comebacked = True
                
            # I＝1 で開始が、総感染者数 I = 0 つまり「感染が収束」した場合
            # 感染収束時の時間の index を subsided に記憶させる.
            # if I == 0 and subsided is None:
            if I == 0:
                print("264_i=",i,"indices=",indices," len=",len(indices),"\n")
                subsided = i
                print("266_I=0_感染収束時間_index=",i,"\n")
                print("267_Simulating SIR 時間_index=%d (S: %d, I: %d, R: %d)" % (i, S, I, R),"\n")
            if comebacked:
                print("269_Simulating SIR 時間_index=%d (S: %d, I: %d, R: %d) (Comebacked)" % (i, S, I, R,"\n"))
            # else:
                # print("Simulating SIR 時間index=%d (I: %d, S: %d, R: %d)" % (i, I, S, R))
            
            
            # それぞれの数値計算結果を記憶.
            self.times = np.append(self.times, i * dt)
            self.zeros = np.append(self.zeros, 0)
            self.S = np.append(self.S, S)
            self.I = np.append(self.I, I)
            self.R = np.append(self.R, R)
            
            # 感染収束時の時間の index が subsided に記憶されており,
            # 時間の index（= i）が subsided とバッファの和になったとき,
            # シミュレーションを終了する.
            
            i# f subsided is not None and i == subsided + buff:
            if I == 0:
                print("self.times=",self.times,"\n")
                print("self.zeros=",self.zeros,"\n")
                print("self.S=",self.S,"\n")
                print("self.I=",self.I,"\n")
                print("self.R=",self.R,"\n")
                self.end = i
                break

    def animate(self):
        
        """
        SIR のシミュレーション & グラフのアニメーションを作成.
        """
        def simulate(i):
            """
            時間の index が i の時のシミュレーションフレームを描画.

            Parameters
            ----------
            i: int
                時間の index.
            """
            print("Animating simulation %1.2f%% (%d / %d)" % ((i + 1) * 100 / self.end, i + 1, self.end))
            plt.cla()
            
            # 目盛りの数値 & 目盛りを消去.
            plt.tick_params(labelbottom=False,
                            labelleft=False,
                            labelright=False,
                            labeltop=False,
                            bottom=False,
                            left=False,
                            right=False,
                            top=False)
            
            ax = fig.add_subplot()
            
            # 正方形 Box の枠の色は白色で設定.
            ax.spines['top'].set_color('white')
            ax.spines['bottom'].set_color('white')
            ax.spines['left'].set_color('white')
            ax.spines['right'].set_color('white')
            
            # x の範囲は -half ~ half まで.
            ax.set_xlim(-half, half)
            # y の範囲は -half ~ half まで.
            ax.set_ylim(-half, half)
            
            # アスペクト比は等しく設定.
            ax.set_aspect('equal')
            
            # 背景色は黒で設定.
            ax.set_facecolor('black')
            
            # 各人間を, 感染が始まっていない場合には緑色の粒子で,
            # 感染している場合には赤色の粒子で,
            # 免疫獲得している場合には青色の粒子で,
            # 時間の index が i の時の各人間の座標の位置に描画.
            for person in self.persons:
                if i < person.infection_start:
                    # この時は未感染状態.
                    circ = patches.Circle(xy=tuple(person.coords[i]),
                                          radius=radius,
                                          fc='lime')
                elif person.infection_start <= i <= person.infection_start + time_to_recover / dt:
                    # この時は感染状態.
                    circ = patches.Circle(xy=tuple(person.coords[i]),
                                          radius=radius,
                                          fc='red')
                    # オーラを周期的に描画.
                    p_aura = 10 * (i - person.infection_start) * dt / time_to_recover
                    aura_radius = 2.5 * (p_aura - int(p_aura)) * radius
                    aura = patches.Circle(xy=tuple(person.coords[i]),
                                          radius=aura_radius,
                                          ec='red',
                                          fill=False)
                    ax.add_patch(aura)
                else:
                    # この時は免疫獲得状態.
                    circ = patches.Circle(xy=tuple(person.coords[i]),
                                          radius=radius,
                                          fc='blue')
                ax.add_patch(circ)
                
        simula_name = "simula_" + str(self.number) + "_" + str(self.R0)
        
        if self.p is not None:
            simula_name += "_p=" + str(self.p)
        if self.comeback:
            simula_name += "_comeback"
        index = 0
        
        while True:
            path = os.path.join(
                this_dir,
                simula_name + "_" + str(index)
            ) + ".mp4"
            if not os.path.exists(path):
                simula_name += "_" + str(index)
                break
            else:
                index += 1
                
        # シミュレーションのアニメーションを作成.
        animate(update=simulate,
                end=self.end,
                name=simula_name)
        
        def draw_graph(i):
            """
            時間の index が i の時のグラフフレームを描画.

            Parameters
            ----------
            i: int
                時間の index.
            """
            print("Animating graph %1.2f%% (%d / %d)" % ((i + 1) * 100 / self.end, i + 1, self.end))
            plt.cla()
            # x 軸, y 軸の目盛りの値 & 目盛りは描画.
            plt.tick_params(labelbottom=True,
                            labelleft=True,
                            labelright=False,
                            labeltop=False,
                            bottom=True,
                            left=True,
                            right=False,
                            top=False)
            ax = fig.add_subplot()
            ax.spines['top'].set_color('white')
            ax.spines['bottom'].set_color('white')
            ax.spines['left'].set_color('white')
            ax.spines['right'].set_color('white')
            
            # x 軸目盛りの色は白で設定.
            ax.tick_params(axis='x', colors='white')
            # y 軸目盛りの色は白で設定.
            ax.tick_params(axis='y', colors='white')
            
            # アスペクト比は自動設定.
            ax.set_aspect('auto')
            ax.set_facecolor('black')
            
            # 時間の index が i までの時間の配列,
            # ゼロ配列,
            # 感染者数の推移の配列,
            # 未感染者数の推移の配列,
            # 免疫獲得者数の推移の配列,
            # をスライスで取得.
            times = self.times[:i + 1]
            zeros = self.zeros[:i + 1]
            I = self.I[:i + 1]
            S = self.S[:i + 1]
            R = self.R[:i + 1]
            
            # それぞれの推移を描画.
            ax.fill_between(times,
                            I + S,
                            I + S + R,
                            facecolor='blue',
                            alpha=1.0,
                            label=str(int(self.R[i])) + "（免疫獲得者）")
            ax.fill_between(times,
                            I,
                            I + S,
                            facecolor='lime',
                            alpha=1.0,
                            label=str(int(self.S[i])) + "（未感染者）")
            ax.fill_between(times,
                            zeros,
                            I,
                            facecolor='red',
                            alpha=1.0,
                            label=str(int(self.I[i])) + "（感染者）")
            ax.legend(loc="upper left", prop={"family": "MS Gothic"})
            
            
            
        graph_name = "graph_" + str(self.number) + "_" + str(self.R0)
                
        if self.p is not None:
            graph_name += "_p=" + str(self.p)
        if self.comeback:
            graph_name += "_comeback"
            
        graph_name += "_" + str(index)
        
        # グラフの推移アニメーションを作成.＿def_animate＿呼出
        # animate(update=draw_graph ,end=self.end ,name=graph_name)


def animate(update,end,name):
    
    """
    matplotlib.animation.FuncAnimation によるアニメーション作成.
    ffmpeg をインストールしておく必要がある.

    Parameters
    ----------
    update: func
        各フレームを描画する関数を指定.
    end: int
        終了フレーム.
    name: str
        アニメーション保存時の名前.
    """
    anim = animation.FuncAnimation(fig,
                                   update,
                                   frames=end,
                                   interval=100/3)
    anim.save(name + ".mp4",
              writer='ffmpeg',
              dpi=300,
              savefig_kwargs={'facecolor':'black'})


if __name__ == "__main__":
    population = Population(number=3,
                            comeback=True,
                            p=0.8)
    population.simulate_sir()
    # population.animate()
    

R0: 0.025456
Re: 0.006778
self.persons= [<__main__.Person object at 0x0000025DA1110D30>, <__main__.Person object at 0x0000025DA1110460>, <__main__.Person object at 0x0000025DA1110A90>] 

init_coords= [array([0., 0.]), array([ 3.62560299, -2.39251081]), array([ 3.54668188, -0.21001856])] 

182_i= 0 indices= [0, 1, 2]  len= 3 


186_i= 0 k= 0   indices= [1, 2] len(indices)= 2
187_i= 0 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [0.00832185 0.00554498] other.co= [ 3.63252591 -2.39972698] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [0.00832185 0.00554498] other.co= [ 3.54879756 -0.20024493] 


186_i= 0 k= 1   indices= [2] len(indices)= 1
187_i= 0 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 3.63252591 -2.39972698] other.co= [ 

187_i= 20 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 3.77098441 -2.54405039] other.co= [ 3.59111129 -0.0047723 ] 


186_i= 20 k= 2   indices= [] len(indices)= 0
187_i= 20 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 21 k= 0   indices= [1, 2] len(indices)= 2
187_i= 21 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [0.18308064 0.12198967] other.co= [ 3.77790733 -2.55126656] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [0.18308064 0.12198967] other.co= [3.59322698 0.00500133] 


186_i= 21 k= 1   indices= [2] len(indices)= 1
187_i= 21 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 3

195_person.co [0.2995865  0.19961946] other.co= [ 3.87482828 -2.65229295] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [0.2995865  0.19961946] other.co= [3.62284659 0.14183217] 


186_i= 35 k= 1   indices= [2] len(indices)= 1
187_i= 35 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 3.87482828 -2.65229295] other.co= [3.62284659 0.14183217] 


186_i= 35 k= 2   indices= [] len(indices)= 0
187_i= 35 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 36 k= 0   indices= [1, 2] len(indices)= 2
187_i= 36 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [0.30790835 0.20516444] other.co= [ 3.8817512  -2.65950912] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [0.30790835 0

195_person.co [0.46602345 0.31051916] other.co= [3.66516031 0.3373048 ] 


186_i= 55 k= 1   indices= [2] len(indices)= 1
187_i= 55 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 4.01328677 -2.79661635] other.co= [3.66516031 0.3373048 ] 


186_i= 55 k= 2   indices= [] len(indices)= 0
187_i= 55 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 56 k= 0   indices= [1, 2] len(indices)= 2
187_i= 56 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [0.47434529 0.31606414] other.co= [ 4.02020969 -2.80383252] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [0.47434529 0.31606414] other.co= [3.667276   0.34707843] 


186_i= 56 k= 1   indices= [2] len(indices)= 1
187_i= 56 k= 1 ,_self.persons[k] = <__main__.Person

186_i= 72 k= 2   indices= [] len(indices)= 0
187_i= 72 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 73 k= 0   indices= [1, 2] len(indices)= 2
187_i= 73 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [0.6158167  0.41032889] other.co= [ 4.13789941 -2.92650742] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [0.6158167  0.41032889] other.co= [3.70324267 0.51323016] 


186_i= 73 k= 1   indices= [2] len(indices)= 1
187_i= 73 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 4.13789941 -2.92650742] other.co= [3.70324267 0.51323016] 


186_i= 73 k= 2   indices= [] len(indices)= 0
187_i= 73 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 74 k= 0   indices= [1, 2] len(indic

...

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [1.17338046 0.78184288] other.co= [3.84499366 1.16806347] 


186_i= 140 k= 1   indices= [2] len(indices)= 1
187_i= 140 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 4.60173537 -3.40999083] other.co= [3.84499366 1.16806347] 


186_i= 140 k= 2   indices= [] len(indices)= 0
187_i= 140 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 141 k= 0   indices= [1, 2] len(indices)= 2
187_i= 141 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [1.18170231 0.78738787] other.co= [ 4.60865829 -3.417207  ] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [1.18170231 0.78738787] other.co= [3.84710935 1.1778371 ] 


186_i= 141 k= 1   indi

195_person.co [1.34813925 0.89828757] other.co= [3.88942307 1.37330973] 


186_i= 161 k= 1   indices= [2] len(indices)= 1
187_i= 161 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 4.74711679 -3.5615304 ] other.co= [3.88942307 1.37330973] 


186_i= 161 k= 2   indices= [] len(indices)= 0
187_i= 161 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 162 k= 0   indices= [1, 2] len(indices)= 2
187_i= 162 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [1.3564611  0.90383255] other.co= [ 4.75403971 -3.56874657] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [1.3564611  0.90383255] other.co= [3.89153876 1.38308336] 


186_i= 162 k= 1   indices= [2] len(indices)= 1
187_i= 162 k= 1 ,_self.persons[k] = <__main_

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [1.50625435 1.00364228] other.co= [ 4.87865236 -3.69863764] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [1.50625435 1.00364228] other.co= [3.92962112 1.55900872] 


186_i= 180 k= 1   indices= [2] len(indices)= 1
187_i= 180 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 4.87865236 -3.69863764] other.co= [3.92962112 1.55900872] 


186_i= 180 k= 2   indices= [] len(indices)= 0
187_i= 180 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 181 k= 0   indices= [1, 2] len(indices)= 2
187_i= 181 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [1.5145762  1.00918727] other.co= [ 4.88557528 -3.70585381] 

193_self.persons[ 2 ]

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [1.63940391 1.09236204] other.co= [ 4.98941915 -3.81409636] 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [1.63940391 1.09236204] other.co= [3.9634721  1.71538682] 


186_i= 196 k= 1   indices= [2] len(indices)= 1
187_i= 196 k= 1 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110460> 

193_self.persons[ 2 ] <__main__.Person object at 0x0000025DA1110A90>
195_person.co [ 4.98941915 -3.81409636] other.co= [3.9634721  1.71538682] 


186_i= 196 k= 2   indices= [] len(indices)= 0
187_i= 196 k= 2 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110A90> 


186_i= 197 k= 0   indices= [1, 2] len(indices)= 2
187_i= 197 k= 0 ,_self.persons[k] = <__main__.Person object at 0x0000025DA1110D30> 

193_self.persons[ 1 ] <__main__.Person object at 0x0000025DA1110460>
195_person.co [1.64772575 1.09790703] other.co= [ 4.99634208 -3.82131253] 

193_self.persons[ 2 ]

<Figure size 288x288 with 0 Axes>

In [ ]:
# pop
# メソッドを実行すると、元のリストや辞書が変わる
# 要素が削除される
# popメソッドは、返り値として、削除した要素を返す。

In [ ]:
"""
output------

R0: 1.697056
Re: 0.451882

print( infected) ⇒ <__main__.Person object at 0x0000026071AEC4C0>

print( Person)   ⇒ <class '__main__.Person'>

Simulating SIR 1 (I: 1, S: 199, R: 0)

<ipython-input-1-fe8463164953>:283: MatplotlibDeprecationWarning:
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.
In a future version, a new instance will always be created and returned.
Meanwhile,this warning can be suppressed, and the future behavior ensured,
by passing a unique label to each axes instance.
  ax = fig.add_subplot()
<ipython-input-1-fe8463164953>：283：MatplotlibDeprecationWarning：
以前の軸と同じ引数を使用して軸を追加すると、
現在、以前のインスタンスが再利用されます。
将来のバージョンでは、新しいインスタンスが常に作成されて返されます。
一方、各軸インスタンスに一意のラベルを渡すことで、この警告を抑制し、
将来の動作を保証できます。
   ax = fig.add_subplot（）

Simulating SIR 662 (I: 0, S: 198, R: 2)
Animating simulation 0.15% (1 / 662)

Animating graph 100.00% (662 / 662)


"""